<a href="https://colab.research.google.com/github/FG2511/ARE/blob/master/model1%2BTN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'''
@File name: model1.ipynb
@Created on 2018-12-20
@Authors: Federica Gerina, Francesca Moi, Silvia Maria Massa
@Description: Given a time-series dataset that contains minute-by-minute data 
about different kind of gases, collected by the uHoo air quality sensor, train
a NN that classifies if a minute belongs to the class "Pasto" (1) otherwise to
the class "Other" (0).
'''

!pip install liac-arff

import arff
import numpy as np
from keras import optimizers
from keras.callbacks import EarlyStopping
from sklearn.utils import compute_class_weight

import sys
sys.path.append('local_modules')

import mlp
import postprocessing_sw
import cooking_inst_mod
import utils

#fix random seed for reproducibility
seed = 5
np.random.seed(seed)

  Stored in directory: /root/.cache/pip/wheels/d1/6a/e7/529dc54d76ecede4346164a09ae3168df358945612710f5203
Successfully built liac-arff


Using TensorFlow backend.


In [2]:
#@title CHOOSE

'''
@Description: MAIN
'''

#LOAD DATA
print("Loading data...")

dataset = '/root/data/uHooComplete_featureDataset_Past_Only.arff' #@param {type:"string"}

with open (dataset, encoding='utf-8') as f:
  dataDictionary = arff.load(f)

data = np.array(dataDictionary['data'])
print("DATASET LOADED")

#CONVERTING VALUES
print("\nConverting values...")
for i in data:
  if(i[-1] == 'Other'): i[-1] = 0
  elif(i[-1] == 'Pasto') : i[-1] = 1

dataset = data.astype('float32')
print("CONVERSION DONE")

#SPLIT INTO INPUT (X) AND OUTPUT (Y) VARIABLES
s = dataset.shape[-1]

X = dataset[:,0:s-1]
Y = dataset[:,s-1]

n_features = s-1

#SPLIT INTO TRAINING, VALIDATION AND TEST SETS
print("\nSplit into training, validation and test sets...")

train_rate = 80
val_rate = 10
train = round(int((dataset.shape[0]*train_rate)/100))
val = round(int((dataset.shape[0]*(train_rate+val_rate))/100))

train_data = X[:train]
train_label = Y[:train]

val_data = X[train+1:val]
val_label = Y[train+1:val]

test_data = X[val+1:]
test_label = Y[val+1:]
print("DATASET SPLITTED")

#COMPUTE CLASS WEIGHT
labels = np.unique(train_label)
classWeight = compute_class_weight('balanced', labels, train_label)
classWeight = dict(zip(labels,classWeight))

#GENERATE MODEL
print("\nGenerate model...")
model = mlp.generate_model_leaky(train_data.shape[-1], n_features)

#OPTIMIZERS
adm = optimizers.Adam(lr=0.0001)

#COMPILE MODEL
print("\nCompile model...")
model.compile(loss='binary_crossentropy', optimizer = adm , metrics=['accuracy'])

#EARLY STOPPING
es = EarlyStopping(monitor='val_loss', min_delta=0, patience=2, verbose=0, mode='auto')

#FIT MODEL
print("\nFit model...")
history = model.fit(train_data, train_label, epochs=10, validation_data = (val_data, val_label), batch_size = 128, shuffle = True, class_weight = classWeight, verbose=1, callbacks = [es])

#EVALUATE MODEL
print("\nEvaluate model...")
scores_test = model.evaluate(test_data, test_label, batch_size=128, verbose = 1)
print("Test loss: %.2f%%" % (scores_test[0] * 100))
print("Test accuracy: %.2f%%" % (scores_test[1] * 100))

#CALCULATE PREDICTIONS
print("\nCalculate predictions...")
pred = model.predict_classes(test_data, batch_size=128, verbose=0)
flat_pred = [item for sublist in pred for item in sublist]

#CONFUSION MATRIX BEFORE POST PROCESSING
print("\n\nCompute confusion matrix BEFORE POST PROCESSING...")
utils.compute_metrics(test_label, flat_pred)

#STORE DATETIME
time = []
for i in test_data:
  time.append(i[-5])

Loading data...
DATASET LOADED

Converting values...
CONVERSION DONE

Split into training, validation and test sets...
DATASET SPLITTED

Generate model...
Instructions for updating:
Colocations handled automatically by placer.

Compile model...

Fit model...
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Train on 280416 samples, validate on 35051 samples
Epoch 1/10
280416/280416 [==============================] - 9s 33us/step - loss: 0.5959 - acc: 0.7245 - val_loss: 0.5980 - val_acc: 0.7076
Epoch 2/10
280416/280416 [==============================] - 8s 27us/step - loss: 0.4938 - acc: 0.8008 - val_loss: 0.5099 - val_acc: 0.7815
Epoch 3/10
280416/280416 [==============================] - 8s 27us/step - loss: 0.4612 - acc: 0.8088 - val_loss: 0.4618 - val_acc: 0.8106
Epoch 4/10
280416/280416 [==============================] - 8s 27us/step - loss: 0.4495 - acc: 0.8070 - val_lo

In [3]:
#POST PROCESSING WITH SLIDING WINDOWS (MINUTE BY MINUTE)
new_pred = postprocessing_sw.sliding_windows(flat_pred,35)

#CONFUSION MATRIX AFTER POST PROCESSING
print("\n\nCompute NEW confusion matrix AFTER POST PROCESSING...")
utils.compute_metrics(test_label, new_pred)



SLIDING WINDOWS FUNCTION...


Compute NEW confusion matrix AFTER POST PROCESSING...
TN 25272
FP 7714
FN 137
TP 1929
ACCURACY: 77.60 %
TRUE NEGATIVE RATE (SPECIFICITY): 76.61 %
TRUE POSITIVE RATE (RECALL): 93.37 %
PRECISION: 20.00 %
F1 SCORE: 32.95 %


In [0]:
def get_precision_recall_f1(p, r): 
  index_pred = []
  index_real = []

  prediction_meals = []
  real_meals = []

  #p = flat_pred
  #r = test_label

  #COSTRUZIONE LISTE DI INDICI

  #Una lista di pasti predetti e una di pasti reali

  for i in range(0, len(p)-1):
    if p[i] == 1:
      index_pred.append(i)
      if (p[i+1] == 0): 
        if(len(index_pred)>5):        
          prediction_meals.append(index_pred)
        index_pred = []
    if r[i] == 1:
      index_real.append(i)
      if (r[i+1] == 0): 
        real_meals.append(index_real)
        index_real = []

  #Una lista di other predetti e una di other reali
  prediction_others = []
  real_others = []

  index_pred = []
  index_real = []

  for i in range(0, len(p)):
    if p[i] == 0:
      index_pred.append(i)
      if ((i+1) in range(0, len(p)) and p[i+1] == 1) or (i+1) not in range(0, len(p)): 
        prediction_others.append(index_pred)
        index_pred = []
    if r[i] == 0:
      index_real.append(i)
      if ((i+1) in range(0, len(p)) and r[i+1] == 1) or (i+1) not in range(0, len(p)): 
        real_others.append(index_real)
        index_real = []
 
  
  
  #Ricerca delle intersezioni, e quindi dei true positive e dei false positive

  intersection = [] #contiene 1 se per un pasto reale j è stata già segnata un'intersezione 
  tp = 0
  fp = 0

  for i in range(0, len(real_meals)): #inizializzazione a 0 per l'array di intersezioni
    intersection.append(0)

  for i in range(0, len(prediction_meals)): #per ogni pasto predetto
    flag_found = 0 #per tenere conto se l'intersezione è già stata trovata all'interno dei pasti reali
    count_int = 0 #tiene conto del numero di intersezioni che un pasto predetto ha con i pasti reali (per il caso in cui un pasto pred intersechi due pasti reali)

    for j in range(0, len(real_meals)):# per ogni pasto reale    
      flag_visited = 0 #tiene conto se un pasto di prediction meals è già stato analizzato (per il caso in cui due pasti pred intersechino un unico pasto reale)

      for x in prediction_meals[i]: #per ogni minuto di pasto controllo se esiste un'intersezione dentro real meals

        if x in real_meals[j]: #se c'è l'intersezione        

          if intersection[j]==0: #ciò significa che non è mai stata trovata un intersezione per il pasto reale j-esimo

            if count_int == 0: #controlla che non siano ancora state trovate intersezioni  
              count_int = count_int + 1
              intersection[j] = 1
              flag_visited = 1 #imposta il flag a uno per indicare che il pasto reale è stato già analizzato e per non contare più di una volta il tp trovato
              flag_found = 1 #imposta il flag a uno per indicare che il pasto predetto è stato trovato nei pasti reali
              tp=tp+1

            elif count_int > 0: #caso in cui un pasto pred interseca due pasti reali e quindi un tp è già stato trovato in precedenza (produce un falso positivo e un vero positivo)           
              intersection[j] = 1
              flag_visited = 1 #imposta il flag a uno per indicare che il pasto reale è stato già analizzato e per non contare più di una volta il tp e il fp trovati
              flag_found = 1            
              fp=fp+1
              tp=tp+1

          elif flag_visited == 0: #caso in cui l'intersezione per il pasto reale j-esimo è già stata trovata ma tale pasto non è ancora stato analizzato con il pasto predetto i-esimo
                                  #risolve il caso in cui più pasti pred intersecano un unico pasto reale
            flag_found = 1
            flag_visited = 1
    #end for j

    if flag_found == 0: #se l'intersezione nn è stata trovata nei pasti reali il pasto predetto è falso positivo
      fp=fp+1
    
    
  #Ricerca falsi negativi

  fn = 0

  for i in range(0, len(real_meals)): #per ogni pasto reale
    j=0
    flag_found = 0
    while j in range(0, len(prediction_meals)) and flag_found==0: # per ogni pasto predetto finchè non viene trovato tra questi l'intersezione con il pasto i-esimo

      for x in real_meals[i]: #per ogni minuto di pasto controllo se esiste un'intersezione dentro prediction_meals

        if x in prediction_meals[j]: #se c'è l'intersezione        
          flag_found = 1 #imposta il flag a uno per indicare che il pasto reale è stato trovato nei pasti predetti

      j = j + 1  
    #end while j

    if flag_found == 0: #se l'intersezione del pasto reale non è stata trovata nei pasti predetti siamo in corrispondenza di un falso negativo
      fn=fn+1
      
      
      
  #Ricerca intersezioni tra other (true negative)

  tn=0
  print(real_others)
  print(prediction_meals)
  for i in range(0, len(real_others)): #per ogni pasto reale
    j=0
    flag_found = 0
    while j in range(0, len(prediction_meals)) and flag_found==0: # per ogni pasto predetto finchè non viene trovato tra questi l'intersezione con il pasto i-esimo

      for x in real_others[i]: #per ogni minuto di pasto controllo se esiste un'intersezione dentro prediction_meals

        if x in prediction_meals[j]: #se c'è l'intersezione        
          flag_found = 1 #imposta il flag a uno per indicare che il pasto reale è stato trovato nei pasti predetti

      j = j + 1  
    #end while j
    print(j)
    if flag_found == 0: #se l'intersezione del pasto reale non è stata trovata nei pasti predetti siamo in corrispondenza di un falso negativo
      tn=tn+1
    



  print("N° pasti reali:", len(real_meals))
  print("N° other reali:", len(real_others))
  print("N° pasti predetti:", len(prediction_meals))
  print("N° other predetti:", len(prediction_others))

  print("TP:", tp)
  print("FP:", fp)
  print("FN:", fn)
  print("TN:", tn)
  

  somma = tp+fp
  precision = (tp/(tp+fp))*100
  TPR = (tp/(tp+fn))*100
  TNR = (tn/(tn+fp))*100
  F1 = 2*((precision*TPR)/(precision+TPR))
  acc = (tp+tn)/(tp+tn+fp+fn)*100
  print("Recall: %.2f %%" % TPR)
  print("Specificity: %.2f %%" % TNR)
  print("Accuracy: %.2f %%" % acc)
  
  print("Precision: %.2f %%" % precision)
  print("F1 score: %.2f %%" % F1)

In [13]:
#COOKING INSTANCE MODALITY

#before post-processing
print("\nCOOKING INSTANCE MODALITY BEFORE POST PROCESSING")
#cooking_inst_mod.get_precision_recall_f1(flat_pred,test_label)
get_precision_recall_f1(flat_pred,test_label)
#after post-processing
print("\nCOOKING INSTANCE MODALITY AFTER POST PROCESSING")
#cooking_inst_mod.get_precision_recall_f1(new_pred,test_label)
get_precision_recall_f1(new_pred,test_label)


COOKING INSTANCE MODALITY BEFORE POST PROCESSING
1
3
8
13
17
19
20
22
24
28
29
33
35
37
39
42
44
46
46
49
52
55
58
62
63
64
67
75
76
82
88
90
93
95
98
99
104
111
113
114
116
125
127
132
135
139
147
150
150
153
160
161
162
166
170
174
177
182
184
184
188
197
199
200
202
210
212
212
213
215
216
218
225
226
229
233
234
238
241
244
248
251
N° pasti reali: 81
N° other reali: 82
N° pasti predetti: 265
N° other predetti: 907
TP: 77
FP: 179
FN: 4
TN: 0
Recall: 95.06 %
Specificity: 0.00 %
Accuracy: 29.62 %
Precision: 30.08 %
F1 score: 45.70 %

COOKING INSTANCE MODALITY AFTER POST PROCESSING
1
3
5
8
10
11
12
13
14
16
17
21
23
25
27
30
31
33
33
35
36
38
39
40
41
42
46
49
50
54
57
58
60
62
64
65
68
72
73
74
75
78
80
82
83
86
89
91
91
92
95
96
97
98
100
103
106
108
109
109
111
116
117
117
118
121
123
123
124
124
125
126
128
129
132
134
135
137
139
142
146
147
N° pasti reali: 81
N° other reali: 82
N° pasti predetti: 156
N° other predetti: 159
TP: 75
FP: 87
FN: 6
TN: 0
Recall: 92.59 %
Specificity: 0